In [2]:
(20 + 1 * 20 - 15 * 1) / (-4)

-6.25

In [4]:
psi = -0.934 + 1.184 * 2 - 0.374 * 10
psi

-2.3060000000000005

In [6]:
from numpy import exp

P = round(1 / (1 + exp(-psi)), 3)
P

0.091

In [7]:
from keras.datasets import mnist
(X_train, y_train), (X_pred, y_pred) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [8]:
X_train.shape

(60000, 28, 28)

In [9]:
28 * 28

784

In [10]:
from numpy import reshape

X_train_reshape = reshape(X_train, (60000, 784))
X_train_reshape.shape

(60000, 784)

Доля объясненной дисперсии должна превышать `0.66`

In [17]:
from sklearn.decomposition import PCA

for i in range(20, 23):
  print(f'n_components={i}:', sum(PCA(n_components=i).fit(X_train_reshape).explained_variance_ratio_))

n_components=20: 0.6435765132189885
n_components=21: 0.654235539763368
n_components=22: 0.6643073622367716


In [20]:
M = 22

pca = PCA(n_components=M, svd_solver='full')

X_train_reshape_reduced = pca.fit_transform(X_train_reshape)
X_train_reshape_reduced.shape

(60000, 22)

In [21]:
from sklearn.model_selection import train_test_split

X_tr, X_t, y_tr, y_t = train_test_split(X_train_reshape_reduced, y_train, test_size=0.3, random_state=2)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

forest = RandomForestClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, n_estimators=10, random_state=2)
clf_1 = OneVsRestClassifier(forest).fit(X_tr, y_tr)

In [26]:
from numpy import ndenumerate

class_8_i = [i[0] for i, digit in ndenumerate(y_t) if digit == 8]
len(class_8_i)

1713

In [27]:
from numpy import full

clf_1.score(X_t[class_8_i], full(len(class_8_i), 8)) * len(class_8_i)

1500.0

In [28]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='lbfgs', random_state=2)
clf_2 = OneVsRestClassifier(logreg).fit(X_tr, y_tr)

In [29]:
class_6_i = [i[0] for i, digit in ndenumerate(y_t) if digit == 6]
len(class_6_i)

1775

In [30]:
clf_2.score(X_t[class_6_i], full(len(class_6_i), 6)) * len(class_6_i)

1650.0

In [31]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, random_state=2)
clf_3 = OneVsRestClassifier(tree).fit(X_tr, y_tr)

In [32]:
class_2_i = [i[0] for i, digit in ndenumerate(y_t) if digit == 2]
len(class_2_i)

1814

In [33]:
clf_3.score(X_t[class_2_i], full(len(class_2_i), 2)) * len(class_2_i)

1525.0

In [34]:
import pandas as pd

df_test = pd.read_csv('https://courses.openedu.ru/assets/courseware/v1/25c0abab1076236b7427fac937c5663e/asset-v1:ITMOUniversity+ADVMLAUTXT+spring_2023_ITMO_mag+type@asset+block/pred_for_task.csv')
df_test.head()

,FileName,Label,f0,f1,f2,f3,f4,f5,f6,f7,...,f774,f775,f776,f777,f778,f779,f780,f781,f782,f783
0,file1,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,file2,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,file3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,file4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,file5,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
df_test.iloc[8]

FileName    file9
Label           5
f0              0
f1              0
f2              0
            ...  
f779            0
f780            0
f781            0
f782            0
f783            0
Name: 8, Length: 786, dtype: object

In [47]:
df_test.iloc[7]

FileName    file8
Label           9
f0              0
f1              0
f2              0
            ...  
f779            0
f780            0
f781            0
f782            0
f783            0
Name: 7, Length: 786, dtype: object

In [35]:
X_test = df_test.drop(columns=['FileName', 'Label'])
X_test.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f774,f775,f776,f777,f778,f779,f780,f781,f782,f783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
X_test_reduced = pca.transform(X_test)
X_test_reduced.shape

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(


(30, 22)

In [37]:
y_test_forest = clf_1.predict_proba(X_test_reduced)

In [39]:
round(y_test_forest[11][6], 3)

0.797

In [40]:
y_test_logreg = clf_2.predict_proba(X_test_reduced)

In [51]:
round(y_test_logreg[8][6], 3)

0.767

In [42]:
y_test_tree = clf_3.predict_proba(X_test_reduced)

In [49]:
round(y_test_tree[7][9], 3)

0.999